In [15]:
import requests
import json
import os
import time
import pickle
from pathlib import Path

In [16]:
api_key = "RGAPI-fb98e73c-8e20-47e7-8b53-c240bf3d81d1"

In [17]:
end_point = "league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5"

In [18]:
url = "https://na1.api.riotgames.com/lol/"+ end_point +"?api_key=" + api_key
x = requests.get(url)
print(x)

<Response [200]>


In [19]:
json_object = json.loads(x.text)

json_formatted_str = json.dumps(json_object, indent=2)
players = json_object["entries"]
print((players[0]))
sorted_players = sorted(players, key=lambda k: k['leaguePoints'])
sorted_player_ids = [d['summonerId'] for d in sorted_players]
good_player_path = Path('data/') / 'good_players.pkl'
with open(good_player_path, 'wb') as f:
    pickle.dump(sorted_player_ids, f)

{'summonerId': 'qJN4WiKvFpewnqA0M7qUsvvc44NEydzj6mCPxyj4auzpRnk', 'summonerName': 'c2 meteos', 'leaguePoints': 1352, 'rank': 'I', 'wins': 294, 'losses': 240, 'veteran': True, 'inactive': False, 'freshBlood': False, 'hotStreak': False}


In [20]:
pickle.load(open(good_player_path, 'rb'))[:10]

['WXJH5Ao3EGUYrOtEQDVakA1nANH_dOyl_HsXnuQtAkVPFsw',
 '7KDhpBuKkcKpjch8A4IxQUxM0zfYD9pgrwv5cGKqaeTOV2o',
 '5MrKSOQZoeZNozUPrmPpiAfj6Hj-I6j16jnko2bOSJwY3hPKqH-XWlLNvA',
 'DcSnaqfvF78tCIhxeY726ZLim36eE0YZ75-p0R6N-3yxhPY',
 '_-Mrs1xvXPc-_PKAU4eslVTBHYsSd4_wRrKp4NYHe7TyRe4L',
 'TFADiCNTRi2osEsGaMfSHvzJote_ROi21FIN0WfR8HjgEn4',
 'gn1A_ZK4DYjO1HMS3m-tbm2BEKUpugWThu-ysJElZuGU9ic',
 'TCP0YrNpUDh4YoIe0V2TeLwIHTeqsxVENy_Jxepl7RZVZ_k',
 'DeGMi0Iy-_bqp3r-RGDc2jNgE-hYBO48uUB4c8lwX5oyP90',
 'IBeQRkELJDqU2dSJ2MEhIxj3RxNXsugrz9YiLbsA-toMhXs']

In [21]:
num_matches = 20
requests_per_summoner = 1 + 1 + num_matches # summonerToID + matchList + numMatches
def get_games(summonerName):
    end_point = "/lol/summoner/v4/summoners/by-name/"+(summonerName).replace(" ", "%20")
    url = "https://na1.api.riotgames.com"+ end_point +"?api_key=" + api_key
    x = requests.get(url)
    json_object = json.loads(x.text)
    print(json_object)
    
    puuid = json_object["puuid"]
    data_path = f'data/{puuid}'
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    
    match_end = f"/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count={num_matches}&type=ranked"
    url = "https://americas.api.riotgames.com"+ match_end +"&api_key=" + api_key
    x = requests.get(url)
    print(url, x)
    matchIds = x.json()
    for matchId in matchIds:
        #print(matchIds)
        match = f"/lol/match/v5/matches/{matchId}"
        url = "https://americas.api.riotgames.com"+ match +"?api_key=" + api_key
        match_request = requests.get(url)
        # player_path = f'{data_path}/{matchId}'
        # if not os.path.exists(player_path):
        #     os.makedirs(player_path)
        with open(data_path + "/" + matchId +".json", 'w') as js:
            json.dump(match_request.json(), js)

def get_games_by_id(summonerId):
    puuid = summonerId
    data_path = f'data/{puuid}'
    if not os.path.exists(data_path):
        os.makedirs(data_path)
    
    match_end = f"/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count={num_matches}&type=ranked"
    url = "https://americas.api.riotgames.com"+ match_end +"&api_key=" + api_key
    x = requests.get(url)
    print(url, x)
    matchIds = x.json()
    for matchId in matchIds:
        #print(matchIds)
        match = f"/lol/match/v5/matches/{matchId}"
        url = "https://americas.api.riotgames.com"+ match +"?api_key=" + api_key
        match_request = requests.get(url)
        # player_path = f'{data_path}/{matchId}'
        # if not os.path.exists(player_path):
        #     os.makedirs(player_path)
        with open(data_path + "/" + matchId +".json", 'w') as js:
            json.dump(match_request.json(), js)

In [22]:
get_games(sorted_players[0]["summonerName"])

{'id': 'WXJH5Ao3EGUYrOtEQDVakA1nANH_dOyl_HsXnuQtAkVPFsw', 'accountId': 'aa7C1AsbJ4_0VdrOzAGOAqUftxqCJYH170T56EpZgt5yslI', 'puuid': 'VqWash5nB3YDPk6SUDDPk0gSBz35Yyti3F1ceMKevyT4AVfQMLaiq_ZmLb1icSO0NeR7wQUIU6Hmzw', 'name': 'Pluto3', 'profileIconId': 4158, 'revisionDate': 1652164346437, 'summonerLevel': 590}
https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/VqWash5nB3YDPk6SUDDPk0gSBz35Yyti3F1ceMKevyT4AVfQMLaiq_ZmLb1icSO0NeR7wQUIU6Hmzw/ids?start=0&count=20&type=ranked&api_key=RGAPI-fb98e73c-8e20-47e7-8b53-c240bf3d81d1 <Response [200]>


In [ ]:
for player in sorted_players[:10]:
    print(player)
    get_games(player['summonerName'])
    time.sleep(20)

{'summonerId': 'WXJH5Ao3EGUYrOtEQDVakA1nANH_dOyl_HsXnuQtAkVPFsw', 'summonerName': 'Pluto3', 'leaguePoints': 737, 'rank': 'I', 'wins': 222, 'losses': 193, 'veteran': False, 'inactive': False, 'freshBlood': False, 'hotStreak': False}
{'id': 'WXJH5Ao3EGUYrOtEQDVakA1nANH_dOyl_HsXnuQtAkVPFsw', 'accountId': 'aa7C1AsbJ4_0VdrOzAGOAqUftxqCJYH170T56EpZgt5yslI', 'puuid': 'VqWash5nB3YDPk6SUDDPk0gSBz35Yyti3F1ceMKevyT4AVfQMLaiq_ZmLb1icSO0NeR7wQUIU6Hmzw', 'name': 'Pluto3', 'profileIconId': 4158, 'revisionDate': 1652164346437, 'summonerLevel': 590}
https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/VqWash5nB3YDPk6SUDDPk0gSBz35Yyti3F1ceMKevyT4AVfQMLaiq_ZmLb1icSO0NeR7wQUIU6Hmzw/ids?start=0&count=20&type=ranked&api_key=RGAPI-fb98e73c-8e20-47e7-8b53-c240bf3d81d1 <Response [200]>
{'summonerId': '7KDhpBuKkcKpjch8A4IxQUxM0zfYD9pgrwv5cGKqaeTOV2o', 'summonerName': 'YoungFartBubbles', 'leaguePoints': 769, 'rank': 'I', 'wins': 260, 'losses': 227, 'veteran': False, 'inactive': False, 'freshBlood': T

In [ ]:
data_path = Path('data/')
for summoner_folder in data_path.iterdir():
    if str(summoner_folder.name).startswith('.'):
        continue
    for game_file in summoner_folder:
        with open(game_file, 'r') as f:
            game_json = json.load(f)

            metadata = game_json['metadata']
            participants = metadata['participants']
            for p in participants:
                if not (data_path / p ).exists():
                    get_games_by_id(p)


In [ ]:
# end_point = "/lol/summoner/v4/summoners/by-name/"+(sorted_players[299]["summonerName"]).replace(" ", "%20")
# url = "https://na1.api.riotgames.com"+ end_point +"?api_key=" + api_key
# x = requests.get(url)
# json_object = json.loads(x.text)
# puuid = json_object["puuid"]
# match_end = f"/lol/match/v5/matches/by-puuid/{puuid}/ids"
# url = "https://americas.api.riotgames.com"+ match_end +"?api_key=" + api_key
# x = requests.get(url)
# matchId = x.json()[2]
# match = f"/lol/match/v5/matches/{matchId}"
# url = "https://americas.api.riotgames.com"+ match +"?api_key=" + api_key
# match_request = requests.get(url)

In [ ]:
# match_obj = match_request.json()
# player_num = -1
# for player in range(len(match_obj['metadata']['participants'])):
#     if match_obj['metadata']['participants'][player] == puuid:
#         player_num = player
# print(match_request.json()['info']['participants'][player]['win'])

False


In [71]:
# print(x.json()[0])

NA1_4290593298
